### XGBoost

In [2]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib as mp
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
url_raw = './Dataset/development_dataset.csv'
url_train = 'scales.csv' #traing dataset
url_dict = './Dataset/Data_Dictionary.xlsx' #data dictionary
url_lead = 'itrImputedlead.csv' 
url_lead_raw = './Dataset/leaderboard_dataset.csv'

In [4]:
## making dataframe
df_raw = pd.read_csv(url_raw)
df_train = pd.read_csv(url_train)
df_dict = pd.read_excel(url_dict)
df_lead = pd.read_csv(url_lead)
df_lead_raw = pd.read_csv(url_lead_raw)

In [5]:
df_raw = df_raw.replace(to_replace = '.', value =np.nan)
df_lead_raw = df_lead_raw.replace(to_replace = '.', value =np.nan)

In [6]:
df_train.head()

,Unnamed: 0,FICO,Business Revenue,TSR,Last 6M Avg Util,Income,Last 6M Avg Remit,Last 6M Avg Bal,Internal Revolve,Lending AR% in last 6M,...,Months in Business,preferred_spend_line,total_cards,preferred_bal_line,preavgremit1_3_all,External_rev_rate,Supplementary Cards,isChargeCust,CMV,Assigned Line
0,0,0.052678,0.352363,0.784935,0.610371,0.013736,0.539663,0.546310,0.602782,0.37204,...,0.493103,0.013089,0.022942,0.009853,0.652225,0.624607,0.670784,0.000000,0.149446,Low
1,1,0.411924,0.046913,0.697432,0.008736,0.303597,0.647431,0.633930,0.244478,0.15774,...,0.145534,0.543781,0.022942,0.539310,0.681579,0.648578,0.000000,0.701962,0.705285,High
2,2,0.792935,0.749787,0.085767,0.402600,0.303597,0.278243,0.284126,0.570605,0.37204,...,0.832524,0.432130,0.022942,0.380423,0.251066,0.282626,0.000000,0.000000,0.113386,Medium
3,3,0.300332,0.745787,0.694903,0.755846,0.379658,0.371070,0.322568,0.108738,0.37204,...,0.664108,0.410747,0.022942,0.427424,0.383629,0.750346,0.670784,0.000000,0.680196,Low
4,4,0.425277,0.212242,0.643990,0.619695,0.137710,0.657461,0.660990,0.817201,0.37204,...,0.832524,0.580546,0.640460,0.590135,0.658431,0.269008,0.670784,0.000000,0.177682,High


In [7]:
df_lead_raw.head()

,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9,VAR10,VAR11,VAR12,VAR13,VAR14,VAR15,VAR16,VAR17,VAR18,VAR19,VAR20
0,1,878.823529,0.833333,821.428571,620.835806,5.181818,0.833974,197.500187,NaN,58.632548,9.433609,40.033361,10.000000,2,10.000000,1.212784,1000.00,1,0,48.099828
1,2,891.764706,0.138889,357.142857,614.092215,3.909091,0.595187,197.016843,253.896073,58.632548,10.144612,NaN,5.666667,1,7.600000,1.004907,992.55,0,0,104.018933
2,3,955.294118,0.055556,NaN,611.574748,6.363636,0.918652,197.660051,22.086661,NaN,0.583494,NaN,6.000000,1,6.000000,1.157115,NaN,1,1,120.586919
3,4,831.764706,0.111111,250.000000,617.740617,6.545455,2.383924,200.526288,NaN,48.539365,NaN,NaN,NaN,3,NaN,2.350858,NaN,0,1,168.592083
4,5,957.647059,0.166667,NaN,623.426802,NaN,0.711240,197.151458,5.013668,49.379748,0.255712,NaN,13.333333,1,13.333333,1.171551,NaN,0,1,109.204819


In [8]:
key = df_lead['VAR1']
#key

In [9]:
df_lead.head()

,Unnamed: 0,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9,...,VAR11,VAR12,VAR13,VAR14,VAR15,VAR16,VAR17,VAR18,VAR19,VAR20
0,0,1,878.823529,0.833333,821.428571,620.835806,5.181818,0.833974,197.500187,79.841411,...,9.433609,40.033361,10.000000,2.0,10.000000,1.212784,1000.000000,1.0,0.0,48.099828
1,1,2,891.764706,0.138889,357.142857,614.092215,3.909091,0.595187,197.016843,253.896073,...,10.144612,-52.920376,5.666667,1.0,7.600000,1.004907,992.550000,0.0,0.0,104.018933
2,2,3,955.294118,0.055556,119.194565,611.574748,6.363636,0.918652,197.660051,22.086661,...,0.583494,132.085648,6.000000,1.0,6.000000,1.157115,810.124901,1.0,1.0,120.586919
3,3,4,831.764706,0.111111,250.000000,617.740617,6.545455,2.383924,200.526288,9.139058,...,7.209404,88.819201,11.089148,3.0,11.865819,2.350858,824.198921,0.0,1.0,168.592083
4,4,5,957.647059,0.166667,119.582615,623.426802,15.127501,0.711240,197.151458,5.013668,...,0.255712,125.551716,13.333333,1.0,13.333333,1.171551,782.887923,0.0,1.0,109.204819


In [ ]:
#### Accessing variable from other notebook
%store -r X
%store -r y
X.shape,y.shape

In [ ]:
X_oth = X_res[:,2:]
#X_oth

In [ ]:
y_res

In [12]:
X = df_train.iloc[:, :-1].values
y = df_train.iloc[:, 20].values

#X_lead = df_lead.iloc[:, 2:].values
#X_lead_raw = df_lead_raw.iloc[:, 1:].values

#X_raw = df_raw.iloc[:, 1:-1].values
#y_raw = df_raw.iloc[:, 20].values

In [ ]:
X

In [ ]:
y_res

In [ ]:
# label Encoding
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X = LabelEncoder()
#X[:, 11] = labelencoder_X.fit_transform(X[:, 11])

labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

In [ ]:
y

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
# X_train = sc_X.fit_transform(X_train)
# X_test = sc_X.transform(X_test)
#X_lead = sc_X.transform(X_lead)
X_oth = sc_X.fit_transform(X_oth)
#sc_y = StandardScaler()
#y_train = sc_y.fit_transform(y_train.reshape(-1,1))

In [ ]:
# split data into train and test sets
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [ ]:
#X_lead = sc_X.transform(X_lead)

#### Fitting Model

In [ ]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
#print(model)

In [ ]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [ ]:
y_pred

In [ ]:
predictions

In [ ]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
y_lead2 = model.predict(X_lead_raw)

In [ ]:
y_lead2

In [ ]:
#key = key.tolist()

In [ ]:
y_lead2 = y_lead2.tolist()

In [ ]:
len(key)

In [ ]:
len(y_lead2)

In [ ]:
ylead21 = labelencoder_y.inverse_transform(y_lead2)

In [ ]:
 ylead21

In [ ]:
dfxyz = pd.DataFrame(ylead,key)

In [ ]:
dfxyz.head()

In [ ]:
dfxyz.to_csv('Anonymous_IITRoorkee_5.csv')

In [ ]:
n_count_classes = pd.value_counts(ylead21, sort = True)

n_count_classes.plot(kind = 'bar', rot=0)

plt.title("Credit Card limit")

#plt.xticks(range(2), LABELS)

plt.xlabel("Class")

plt.ylabel("Frequency")

In [ ]:
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers

In [ ]:
col = df_train.columns[3:-1]
len(col)

In [ ]:
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=col)
feat_importances.nlargest(15).plot(kind='barh')
plt.show()